In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os
from config import census_key
import json
import requests 
#from census import Census

In [18]:
# Import DA Job csv
da_job = pd.read_csv("Resources/DataAnalyst.csv")

# Drop unnecessary columns
da_job_data = da_job[["ID", "Job Title", "Salary Estimate", "Job Description", "Rating", "Company Name", "Location", "Headquarters", "Size", 
                                     "Founded", "Type of ownership", "Industry", "Sector", "Revenue", "Competitors", "Easy Apply"]]
# create df
da_job_data_df = pd.DataFrame(da_job_data)

# drop columns
gd_df = da_job_data_df.drop(['Easy Apply'], axis=1)

# export to csv
#gd_df.to_csv ("glassdoor.csv", index = False, header=True)
gd_df

,ID,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy
3,3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1
4,4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248,2248,RQS - IHHA - 201900004460 -1q Data Security An...,$78K-$104K (Glassdoor est.),Maintains systems to protect data from unautho...,2.5,"Avacend, Inc.\n2.5","Denver, CO","Alpharetta, GA",51 to 200 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,Unknown / Non-Applicable,-1
2249,2249,Senior Data Analyst (Corporate Audit),$78K-$104K (Glassdoor est.),Position:\nSenior Data Analyst (Corporate Audi...,2.9,Arrow Electronics\n2.9,"Centennial, CO","Centennial, CO",10000+ employees,1935,Company - Public,Wholesale,Business Services,$10+ billion (USD),"Avnet, Ingram Micro, Tech Data"
2250,2250,"Technical Business Analyst (SQL, Data analytic...",$78K-$104K (Glassdoor est.),"Title: Technical Business Analyst (SQL, Data a...",-1.0,Spiceorb,"Denver, CO",-1,-1,-1,-1,-1,-1,-1,-1
2251,2251,"Data Analyst 3, Customer Experience",$78K-$104K (Glassdoor est.),Summary\n\nResponsible for working cross-funct...,3.1,Contingent Network Services\n3.1,"Centennial, CO","West Chester, OH",201 to 500 employees,1984,Company - Private,Enterprise Software & Network Solutions,Information Technology,$25 to $50 million (USD),-1


In [23]:
# Determine top 50 cities with data analyst jobs

# group by city
t50 = gd_df.groupby('Location').count()["ID"]

#create df
t50_df = pd.DataFrame({"gaJobs": t50})

#reset index
t50_df = t50_df.reset_index(drop=False)

# sort data
t50_df = t50_df.sort_values(['gaJobs', 'Location'], ascending=[False, True]).head(50)

# export to csv
t50_df.to_csv ("top50city.csv", index = False, header=True) 

t50_df

,Location,gaJobs
154,"New York, NY",310
47,"Chicago, IL",130
200,"San Francisco, CA",119
13,"Austin, TX",81
133,"Los Angeles, CA",80
44,"Charlotte, NC",78
106,"Houston, TX",72
55,"Dallas, TX",66
198,"San Diego, CA",62
176,"Philadelphia, PA",56


In [26]:
# join 2 tables to filter job table by top 50 city

gd_t50_df = gd_df.merge(t50_df,how='right', left_on='Location', right_on='Location')

# export to csv
gd_t50_df.to_csv ("glassdoorfinal.csv", index = False, header=True)
gd_t50_df


,ID,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,gaJobs
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,310
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,310
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,310
3,3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1,310
4,4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,2213,"Data Analyst 3, Customer Experience",$57K-$100K (Glassdoor est.),Business Unit:SummaryResponsible for working c...,3.6,Comcast\n3.6,"Centennial, CO","Philadelphia, PA",10000+ employees,1963,Company - Public,-1,-1,$10+ billion (USD),"AT&T, Verizon",9
1802,2242,Business Systems Analyst-Data,$78K-$104K (Glassdoor est.),"Location Centennial, CO Description Our client...",3.5,"The Judge Group, Inc.\n3.5","Centennial, CO","Wayne, PA",1001 to 5000 employees,1970,Company - Private,Staffing & Outsourcing,Business Services,$100 to $500 million (USD),"Adecco, Kforce",9
1803,2245,Data Analyst Supporting the DEA #20-242,$78K-$104K (Glassdoor est.),Salary:\nPublished Job Title:\nData Analyst Su...,2.8,Forfeiture Support Associates\n2.8,"Centennial, CO","Ashburn, VA",1001 to 5000 employees,2004,Company - Private,Staffing & Outsourcing,Business Services,$50 to $100 million (USD),-1,9
1804,2249,Senior Data Analyst (Corporate Audit),$78K-$104K (Glassdoor est.),Position:\nSenior Data Analyst (Corporate Audi...,2.9,Arrow Electronics\n2.9,"Centennial, CO","Centennial, CO",10000+ employees,1935,Company - Public,Wholesale,Business Services,$10+ billion (USD),"Avnet, Ingram Micro, Tech Data",9
